<a href="https://colab.research.google.com/github/JustinRSK/2024_Intro_Python/blob/main/Project/SwissImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests


Download all SwissImage tiles

In [ ]:
import csv
import os
import requests

CSV_FILE = "/content/tiles.csv"
OUT_DIR = "/content/swissimage_tiles"

os.makedirs(OUT_DIR, exist_ok=True)

with open(CSV_FILE, newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        for url in row:
            url = url.strip()
            if not url:
                continue

            filename = url.split("/")[-1]
            out_path = os.path.join(OUT_DIR, filename)

            print("Downloading:", filename)
            r = requests.get(url, stream=True)
            r.raise_for_status()

            with open(out_path, "wb") as out:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        out.write(chunk)

print("✔ ALL FILES DOWNLOADED ✔")


Downloading: swissimage-dop10_2023_2536-1179_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1180_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1181_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1182_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1183_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1184_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1185_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1186_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1187_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1188_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1189_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1190_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1191_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1192_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1193_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1194_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1195_2_2056.tif
Downloading: swissimage-dop10_2023_2536-1196_2_2

In [ ]:
!pip install rasterio


Merge all tiles into ONE raster

In [ ]:
import rasterio
from rasterio.merge import merge
import glob

tiles = glob.glob("/content/swissimage_tiles/*.tif")

src_files = [rasterio.open(t) for t in tiles]
mosaic, out_trans = merge(src_files)

out_meta = src_files[0].meta.copy()
out_meta.update({
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans
})

with rasterio.open("/content/swissimage_merged.tif", "w", **out_meta) as dest:
    dest.write(mosaic)


Clip merged raster to the AOI polygon

In [ ]:
import geopandas as gpd
from rasterio.mask import mask
import rasterio

# Load AOI
aoi = gpd.read_file("/content/AOI_Grande_Caricaie.geojson")
aoi = aoi.to_crs("EPSG:2056")  # ensure CRS

merged_path = "/content/swissimage_merged.tif"
clipped_path = "/content/SwissImage_AOI_clipped.tif"   # <--- DEFINE IT HERE

# Clip raster
with rasterio.open(merged_path) as src:
    clipped, transform = mask(src, aoi.geometry, crop=True)
    out_meta = src.meta.copy()
    out_meta.update({
        "height": clipped.shape[1],
        "width": clipped.shape[2],
        "transform": transform
    })

# Save clipped raster
with rasterio.open(clipped_path, "w", **out_meta) as dest:
    dest.write(clipped)

print("✔ Clip saved to:", clipped_path)


✔ Clip saved to: /content/SwissImage_AOI_clipped.tif


In [ ]:
from google.colab import files
files.download(clipped_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>